# 2D potential equation

In [ ]:
import numpy as np
from numpy.linalg import solve
import matplotlib.pyplot as plt
from time import time

# for sparse matrices
from scipy.sparse import csc_matrix # for sparse matrix
from scipy.sparse.linalg import spsolve # for sparse matrix

from mesh_utils import rectangular_mesh # import everything from custom module mesh_utils

%matplotlib notebook

In [ ]:
#################
# USER SETTINGS #
#################

nx = 100; lx = 50 # number of points and length of domain in x direction
ny = 40; ly = 20 # number of points and length of domain in y direction

u_out = 1 # velocity at outflow
u_in = u_out # velocity at inflow

ix_phi = nx - 2 # x-coordinate at which potential is provided
iy_phi = 1 # y-coordinate at which potential is provided
phi_default = 100 # arbitrary value of potential 

Since we already have a class `rectangular_mesh` taking care about the mesh generation, we can just exploit it; however, such class still does not contain information about where inflow and outflow sections are located.

This is fine: such information is specific to this problem - it wouldn't make sense to include it into a class `rectangular_mesh` that is suit for a variety of different problems! However, what could be done is creating a _child class_ of `rectangular_mesh`, meaning a class that _inherits_ everything that is already contained in `rectangular_mesh` - but also adding new content. We will call the children class `potential_mesh`, and it will contain what we now need - meaning, a description of inflow and outflow; we will refer to `rectangular_mesh` as the _parent_ class. This is done by:

In [ ]:
class potential_mesh(rectangular_mesh):
    
    #########################################################################
    
    def is_inout(self, ik, boundary):
        
        # inout = 'in' if you are on the inflow
        # inout = 'out' if you are on the outflow
        
        # You are being passed the 1d index of the point and the string boundary;
        # you can take advantage of both.
        # You can also calculate the 2d indices if you want.
        # YOU CAN USE ANYTHING THAT WAS DEFINED INSIDE CLASS rectangular_mesh!
            
        return inout

Now, class `potential_mesh` contains everything that was in `rectangular mesh`, with the addition of functions `is_inflow` and `is_outflow`. We have now a complete description of geometry, so we can actually create an instance of the mesh:

In [ ]:
m = potential_mesh(nx, lx, ny, ly)

It is now time for the allocation of the coefficient matrix `A` and known term `b`:

In [ ]:
# allocation
A = np.zeros(#)
b = np.zeros(#)

for ik, _ in enumerate(#): # cycle over rows of A
    
    # Each row of A corresponds to a grid point P (in particular, 
    # the equation contained in a given row is evaluated at point P).
    
    # Any row of A should correspond to the discretised Laplace (or Poisson)
    # equation, but there are exceptions:
    # - if you are on the boundary, you have the non-penetration condition
    #   (which is a homogeneous Neumann condition)
    # - if the piece of boundary you are on is actually an inflow or outflow
    #   section, you have Neumann conditions

Since only Neumann conditions have been supplied, matrix A is currently singular. One has then to specify the value of the potential at an arbitrary point.

In [ ]:
# the settings contain variables ix_phi and iy_phi indicating the position at which
# the value for the potential is provided. You should modify the corresponding row
# of A so that it reads:
#
# phi[ix_phi,iy_phi] = phi_default

# Of course, matrix phi doesn't really exist - but you have the 1D equivalent...
# Also, phi_default is also specified in the user settings.

And now, let's solve the problem:

In [ ]:
start = time()
phi = solve(A,b) # for normal solution
#A = csc_matrix(A) # converts A to a sparse matrix
#phi = spsolve(A,b) # for sparse matrix solution
end = time()
print('Matrix inversion took', end-start, 'seconds to complete.')

__Hint:__ try and switch the datatypes of matrices A and b from numpy.array to scipy.sparse.csc_matrix; then, use solver scipy.sparse.linalg.spsolve. This means, matrices are archived as sparse. How does the computing time change?

## Postprocessing

In [ ]:
# treatment for mesh before plotting
phi = phi.reshape((m.nx, m.ny))
phi = phi.transpose()

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Velocity potential and isocontours')
pos = ax.pcolormesh(m.x, m.y, phi, shading='gouraud')
fig.colorbar(pos)
ax.contour(m.x, m.y, phi, 30, colors='white', linewidths=.7)

In [ ]:
print(phi.shape)
# calculate gradient
v,u = np.gradient(phi, m.dx, m.dy)

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Streamlines')
ax.streamplot(m.x, m.y, u, v, linewidth=np.sqrt(u**2 + v**2)*3) # start_points = np.array([[25, i] for i in np.linspace(0, m.ly, 30)])
plt.xlim(0, m.lx)
plt.ylim(0, m.ly)

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Velocity field')
ax.quiver(m.x, m.y, u, v)
plt.xlim(0, m.lx)
plt.ylim(0, m.ly)